# Multiple infillers

Here we investigate two infillers that perform many cruncher operations. One is designed to break down composits, like the kyoto gases, the other designed to infill all required data. 

## Imports

In [1]:
# NBVAL_IGNORE_OUTPUT
import os.path
import traceback

import pandas as pd
import pyam
import matplotlib.pyplot as plt
import numpy as np

import silicone.multiple_infillers as mi
from silicone.utils import (
    _get_unit_of_variable,
    find_matching_scenarios,
    _make_interpolator,
    _make_wide_db,
    get_sr15_scenarios, 
    return_cases_which_consistently_split,
    convert_units_to_MtCO2_equiv
)


<IPython.core.display.Javascript object>

pyam - INFO: Running in a notebook, setting `pyam` logging level to `logging.INFO` and adding stderr handler


## Constants

In [2]:
SR15_SCENARIOS = "./sr15_scenarios.csv"

## Example data

Here we pull some example data by downloading a selection of the SR1.5 scenarios.

In [3]:
# NBVAL_IGNORE_OUTPUT
valid_model_ids = [
        "MESSAGE*",
        "AIM*",
        "C-ROADS*",
        "GCAM*",
        "IEA*",
        "IMAGE*",
        "MERGE*",
        "POLES*",
        "REMIND*",
        "WITCH*"
    ]
if not os.path.isfile(SR15_SCENARIOS):
    get_sr15_scenarios(SR15_SCENARIOS, valid_model_ids)

### Starting point

Our starting point is the test data, loaded a `pyam.IamDataFrame`.

In [4]:
# NBVAL_IGNORE_OUTPUT
sr15_data = pyam.IamDataFrame(SR15_SCENARIOS)
target = "Emissions|CO2"
constituents = ["Emissions|CO2|*"]
to_infill = sr15_data.filter(model="WITCH*", variable=[target] + constituents)
database = sr15_data.filter(model="WITCH*", keep=False)
to_infill.head()

pyam.utils - INFO: Reading `./sr15_scenarios.csv`


,model,scenario,region,variable,unit,year,meta,subannual,value
231183,WITCH-GLOBIOM 3.1,SSP1-19,World,Emissions|CO2,Mt CO2/yr,2005,0,0.0,31922.044350
231184,WITCH-GLOBIOM 3.1,SSP1-19,World,Emissions|CO2,Mt CO2/yr,2010,0,0.0,35303.387760
231185,WITCH-GLOBIOM 3.1,SSP1-19,World,Emissions|CO2,Mt CO2/yr,2020,0,0.0,37312.307110
231186,WITCH-GLOBIOM 3.1,SSP1-19,World,Emissions|CO2,Mt CO2/yr,2030,0,0.0,10560.008460
231187,WITCH-GLOBIOM 3.1,SSP1-19,World,Emissions|CO2,Mt CO2/yr,2040,0,0.0,7435.934987


### Investigating where the data is consistent

A utility function called return_cases_which_consistently_split indicates which variables consist of only other variables, which is useful to know in order to work out where data can be consistently split using decompose_collection_with_time_dep_ratio. Note that this is not a requirement for using that method (a consistent aggregate value is constructed in any case) but indicates that this is approach is rigorous. 

In the first instance, it does not find any results because there are several layers of constituents. However with only one layer, this works as expected. Note that if only one layer is used, there is also a pyam built-in function called 'check_consistency' that performs the same effect. 

In [5]:
no_cases = return_cases_which_consistently_split(
    to_infill, target, constituents, 
)
len(no_cases)

0

However in the case below, we select only the next level of info, and find it matches in all cases (the number of cases does not depend on the accuracy, as shown in the second box. 

In [6]:
all_cases = return_cases_which_consistently_split(
    to_infill, target, ["Emissions|CO2|AFOLU", "Emissions|CO2|Energy and Industrial Processes"], 
)
len(all_cases)

39

In [7]:
all_cases = return_cases_which_consistently_split(
    to_infill, target, ["Emissions|CO2|AFOLU", "Emissions|CO2|Energy and Industrial Processes"], 
     how_close={
            'equal_nan': True,
            'rtol': 100, # This means that we accept a factor of 100 inaccuracy. 
    }
)
len(all_cases)

39

### Using the infiller functions
Here we show the use of the decompose_collection_with_time_dep_ratio and infill_all_required_emissions_for_openscm functions. 

In [8]:
database.data["subannual"] = 0
database.data["meta"] = 0
database.tail()

,model,scenario,region,variable,unit,year,meta,subannual,value
231123,REMIND-MAgPIE 1.7-3.0,SMP_REF_Sust,World,Emissions|VOC|Other,Mt VOC/yr,2060,0,0,23.2311
231124,REMIND-MAgPIE 1.7-3.0,SMP_REF_Sust,World,Emissions|VOC|Other,Mt VOC/yr,2070,0,0,22.1971
231125,REMIND-MAgPIE 1.7-3.0,SMP_REF_Sust,World,Emissions|VOC|Other,Mt VOC/yr,2080,0,0,21.1632
231126,REMIND-MAgPIE 1.7-3.0,SMP_REF_Sust,World,Emissions|VOC|Other,Mt VOC/yr,2090,0,0,20.1292
231127,REMIND-MAgPIE 1.7-3.0,SMP_REF_Sust,World,Emissions|VOC|Other,Mt VOC/yr,2100,0,0,19.0953


In [9]:
to_infill.data["subannual"] = 0
to_infill.data["meta"] = 0
to_infill.tail()

,model,scenario,region,variable,unit,year,meta,subannual,value
244614,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,Emissions|CO2|Energy|Supply|Electricity,Mt CO2/yr,2060,0,0,29487.271577
244615,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,Emissions|CO2|Energy|Supply|Electricity,Mt CO2/yr,2070,0,0,31404.870008
244616,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,Emissions|CO2|Energy|Supply|Electricity,Mt CO2/yr,2080,0,0,31911.146283
244617,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,Emissions|CO2|Energy|Supply|Electricity,Mt CO2/yr,2090,0,0,30995.012917
244618,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,Emissions|CO2|Energy|Supply|Electricity,Mt CO2/yr,2100,0,0,29852.454850


In [26]:
infilled = mi.InfillAllRequiredVariables(
    to_infill,
    database,
    [target],
    output_timesteps = list(range(2020, 2101, 10))
)

c:\users\rlamboll\constraincode\silicone\src\silicone\multiple_infillers\infill_all_required_emissions_for_openscm.py:162: UserWarning: No data for ['Emissions|HFC|HFC245ca']
  warnings.warn("No data for {}".format(unavailable_variables))
Filling required variables: 100%|██████████████████████████████████████████████████████| 22/22 [00:19<00:00,  1.11it/s]


We now have a complete scenario, with all required variables. 

In [18]:
infilled.variables()

0                                          Emissions|BC
1                                         Emissions|CH4
2                                          Emissions|CO
3                                         Emissions|CO2
4                                   Emissions|CO2|AFOLU
5         Emissions|CO2|Energy and Industrial Processes
6                           Emissions|CO2|Energy|Supply
7               Emissions|CO2|Energy|Supply|Electricity
8                                  Emissions|HFC|HFC125
9                                 Emissions|HFC|HFC134a
10                                Emissions|HFC|HFC143a
11                               Emissions|HFC|HFC227ea
12                                  Emissions|HFC|HFC23
13                               Emissions|HFC|HFC245ca
14                                  Emissions|HFC|HFC32
15                               Emissions|HFC|HFC43-10
16                                        Emissions|N2O
17                                        Emissi

In [21]:
infilled.validate(criteria={'Emissions|BC': {'lo': 0.001, 'year': 2020}})

c:\users\rlamboll\constraincode\silicone\src\silicone\multiple_infillers\infill_all_required_emissions_for_openscm.py:162: UserWarning: No data for ['Not a variable']
  warnings.warn("No data for {}".format(unavailable_variables))
Filling required variables: 100%|████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.94it/s]


,model,scenario,region,variable,unit,year,meta,subannual,value
45,WITCH-GLOBIOM 3.1,SSP1-19,World,Not a variable,Mt Co2-equiv/yr,2020,0,0,0.0
46,WITCH-GLOBIOM 3.1,SSP1-19,World,Not a variable,Mt Co2-equiv/yr,2030,0,0,0.0
47,WITCH-GLOBIOM 3.1,SSP1-19,World,Not a variable,Mt Co2-equiv/yr,2040,0,0,0.0
48,WITCH-GLOBIOM 3.1,SSP1-19,World,Not a variable,Mt Co2-equiv/yr,2050,0,0,0.0
49,WITCH-GLOBIOM 3.1,SSP1-19,World,Not a variable,Mt Co2-equiv/yr,2060,0,0,-0.0
